# A small demo of background generator

[should work in both python2 and python3]


In [1]:
from __future__ import print_function
from prefetch_generator import BackgroundGenerator, background,__doc__

print(__doc__)


#based on http://stackoverflow.com/questions/7323664/python-generator-pre-fetch

This is a single-function package that transforms arbitrary generator into a background-thead generator that prefetches several batches of data in a parallel background thead.

This is useful if you have a computationally heavy process (CPU or GPU) that iteratively processes minibatches from the generator while the generator consumes some other resource (disk IO / loading from database / more CPU if you have unused cores). 

By default these two processes will constantly wait for one another to finish. If you make generator work in prefetch mode (see examples below), they will work in parallel, potentially saving you your GPU time.

We personally use the prefetch generator when iterating minibatches of data for deep learning with tensorflow and theano ( lasagne, blocks, raw, etc.).

This package contains two objects
 - BackgroundGenerator(any_other_generator[,max_prefetch = something])
 - @background([max

In [2]:
###your super-mega data iterator
import numpy as np
import time

def iterate_minibatches(n_batches, batch_size=10):
    for b_i in range(n_batches):
        time.sleep(0.1) #here it could read file or SQL-get or do some math
        X = np.random.normal(size=[batch_size,20])
        y = np.random.randint(0,2,size=batch_size)
        yield X,y

### regular mode

In [3]:
%%time

#tqdm made in china
print('/'+'-'*42+' Progress Bar ' + '-'*42 + '\\')

for b_x,b_y in iterate_minibatches(50):
    #training
    time.sleep(0.1) #here it could use GPU for example
    print('!',end=" ")
    
print()

/------------------------------------------ Progress Bar ------------------------------------------\
! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! 
CPU times: user 100 ms, sys: 20 ms, total: 120 ms
Wall time: 10.1 s


### with prefetch

In [4]:
%%time

print('/'+'-'*42+' Progress Bar ' + '-'*42 + '\\')

for b_x,b_y in BackgroundGenerator(iterate_minibatches(50)):
    #training
    time.sleep(0.1) #here it could use some GPU
    print('!',end=" ")
print()

/------------------------------------------ Progress Bar ------------------------------------------\
! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! 
CPU times: user 68 ms, sys: 16 ms, total: 84 ms
Wall time: 5.14 s


### Same with decorator

In [5]:
###your super-mega data iterator again, now with background decorator
import numpy as np
import time

@background(max_prefetch=3)
def bg_iterate_minibatches(n_batches, batch_size=10):
    for b_i in range(n_batches):
        time.sleep(0.1) #here it could read file or SQL-get or do some math
        X = np.random.normal(size=[batch_size,20])
        y = np.random.randint(0,2,size=batch_size)
        yield X,y


In [6]:
%%time

print('/'+'-'*42+' Progress Bar ' + '-'*42 + '\\')

for b_x,b_y in bg_iterate_minibatches(50):
    #training
    time.sleep(0.1)#you guessed it
    print('!',end=" ")
print()

/------------------------------------------ Progress Bar ------------------------------------------\
! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! 
CPU times: user 56 ms, sys: 20 ms, total: 76 ms
Wall time: 5.14 s
